In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import math
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
from keras.models import Sequential
from keras import regularizers
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.optimizers import RMSprop
from keras.initializers import glorot_uniform, glorot_normal, RandomUniform
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from datetime import timedelta
from sklearn.externals import joblib 
import datetime

In [0]:
def Clean(data,valid = False):
  data = data.values
  new_data = []
  for i in range(len(data)):
    if(str(data[i][0]) == str(1e400*0)):
      pass
    else:
      new_line = list(map(str,data[i][0].split(';')))
      new_data.append(new_line)

  data = pd.DataFrame(new_data,columns=['v0','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','target'])
  data.to_csv("data.csv",index=False)
  data = pd.read_csv("/content/data.csv")

  return data


In [111]:
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/binary_classification/training.csv")
data = Clean(data)
data.head()


,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,target
0,75,"""f""","""t""",1,"""t""","""g""",80.0,5,800000.0,"""t""",0,"""no."""
1,29,"""f""","""f""",0,"""f""","""s""",200.0,0,2000000.0,NaN,0,"""no."""
2,335,"""f""","""f""",0,"""f""","""g""",0.0,120,0.0,NaN,0,"""no."""
3,5,"""f""","""f""",0,"""t""","""g""",232.0,0,2320000.0,"""f""",0,"""no."""
4,5,"""f""","""f""",0,"""t""","""g""",160.0,0,1600000.0,"""f""",0,"""no."""


In [113]:
for i in data:
  if (len(data[i].unique()) < 5):
    print(i,data[i].unique())

v1 ['"f"' '"t"']
v2 ['"t"' '"f"']
v4 ['"t"' '"f"']
v5 ['"g"' '"s"' '"p"']
v9 ['"t"' nan '"f"']
v10 [0 1]
target ['"no."' '"yes."']


In [0]:
def Prepare(data):
  data['v9'] = data['v9'].fillna(2)
  data['v6'] = data['v6'].fillna(0)
  data['v8'] = data['v8'].fillna(0)
  
  for i in range(len(data['v1'].unique())):
    data['v1'] = data['v1'].replace(data['v1'].unique()[i],i)
  
  for i in range(len(data['v2'].unique())):
    data['v2'] = data['v2'].replace(data['v2'].unique()[i],i)

  for i in range(len(data['v4'].unique())):
    data['v4'] = data['v4'].replace(data['v4'].unique()[i],i)

  for i in range(len(data['v5'].unique())):
    data['v5'] = data['v5'].replace(data['v5'].unique()[i],i)

  for i in range(len(data['v9'].unique())):
    data['v9'] = data['v9'].replace(data['v9'].unique()[i],i)

  for i in range(len(data['target'].unique())):
    data['target'] = data['target'].replace(data['target'].unique()[i],i)
  
  return data

In [0]:
data = Prepare(data)

In [116]:
y = data['target']
x = data.drop(['target'],axis=1)
x.shape,y.shape

((2009, 11), (2009,))

In [117]:
scaler = StandardScaler()
x = scaler.fit_transform(x)
x.shape

(2009, 11)

In [0]:
neurons = 512                 
activ_func = 'tanh'  
loss = 'binary_crossentropy'                  
optimizer="adam"              
dropout = 0.05                 

In [119]:
model = Sequential()
model.add(Dense(512,input_dim=np.shape(x)[1],activation=activ_func))
model.add(Dense(256,activation=activ_func))
model.add(Dense(units=1,activation = 'sigmoid'))
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               6144      
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 137,729
Trainable params: 137,729
Non-trainable params: 0
_________________________________________________________________


In [120]:
model.fit(np.array(x), np.array(y), epochs=2, batch_size=32, verbose=1)

Epoch 1/2
2009/2009 [==============================] - 1s 292us/step - loss: 0.0894 - acc: 0.9701
Epoch 2/2
2009/2009 [==============================] - 0s 115us/step - loss: 5.4795e-04 - acc: 1.0000


In [124]:
valid = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/binary_classification/validation.csv")
valid = Clean(valid)
valid = Prepare(valid)
print(valid.head())
valid_y = data['target']
valid_x = data.drop(['target'],axis=1)
valid_x.shape,valid_y.shape

    v0  v1  v2  v3  v4  v5   v6   v7         v8  v9  v10  target
0  585   0   0   0   0   0  420    0  4200000.0   0    1       0
1   54   1   0   0   0   1  136    1  1360000.0   0    0       0
2  585   1   0   0   1   1  240    3  2400000.0   0    1       0
3  125   0   0   0   1   1  120  375  1200000.0   0    0       0
4    4   1   0   0   0   1  120  475  1200000.0   1    1       0


((2009, 11), (2009,))

In [123]:
from sklearn.metrics import precision_recall_fscore_support
y_pred = model.predict_classes(valid_x)
result = precision_recall_fscore_support(valid_y, y_pred,average='binary')
print('precision=',result[0],'recall=',result[1], 'f1score=',result[2])

precision= 0.9238427078148332 recall= 1.0 f1score= 0.9604139715394566
